In [1]:
%matplotlib inline

# Dependencies and Setup
#from stateAbbr import us_state_abbrev
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import datetime 
import gzip
from pandas.io.json import json_normalize
import pprint as pp
import os
import json
import gmaps
import ipywidgets as widget
import csv
#Pip instsall pyzipcode
#from uszipcode import ZipcodeSearchEngine
 

#zc = search.by_zipcode(77065) 
#pp.pprint(zc)
from os import listdir
from os.path import isfile,join
#import filesplit
# used Bash and linux split command to break up file
#split --lines=300000 --numeric-suffixes --suffix-length=3 arcos_all_washpost.tsv aaww
#int 3000000 line segments.


# DEAfilespath ="E:\\RiceProjectData\\DEAData"
# DEAFilelist=[f for f in listdir(DEAfilespath) if isfile(join(DEAfilespath,f))]
# DEAFiledict={}

def Pickupfromcount(num):
    '''
    This sets a dictionary of filenames to 
    to = True, meaning that the files have
    already been processed, and that any thing
    where it is false should be processed
    '''
    cntr=0
    #Keep Track of Processed Files
    for f in DEAFilelist:
        DEAFiledict.update({f:(cntr<num)})
        cntr+=1
    
# bigfile="E:\\RiceProjectData\\DEAData\\aaww001"
# filename0='arcos-tx-statewide-pharmacy.tsv'
# filename1='arcos-tx-statewide-distributor.tsv'
# filename2='arcos-tx-statewide-labeler.tsv'
Outputhdr=['REPORTER_NAME',
 'BUYER_NAME',
 'BUYER_ZIP',
 'BUYER_COUNTY',
 'DRUG_NAME',
 'QUANTITY',
 'UNIT',
 'DOSAGE_UNIT',
 'TRANSACTION_ID',
 'Ingredient_Name', 
 'dos_str']

Masterhdr=['REPORTER_DEA_NO',
 'REPORTER_BUS_ACT',
 'REPORTER_NAME',
 'REPORTER_ADDL_CO_INFO',
 'REPORTER_ADDRESS1',
 'REPORTER_ADDRESS2',
 'REPORTER_CITY',
 'REPORTER_STATE',
 'REPORTER_ZIP',
 'REPORTER_COUNTY',
 'BUYER_DEA_NO',
 'BUYER_BUS_ACT',
 'BUYER_NAME',
 'BUYER_ADDL_CO_INFO',
 'BUYER_ADDRESS1',
 'BUYER_ADDRESS2',
 'BUYER_CITY',
 'BUYER_STATE',
 'BUYER_ZIP',
 'BUYER_COUNTY',
 'TRANSACTION_CODE',
 'DRUG_CODE',
 'NDC_NO',
 'DRUG_NAME',
 'QUANTITY',
 'UNIT',
 'ACTION_INDICATOR',
 'ORDER_FORM_NO',
 'CORRECTION_NO',
 'STRENGTH',
 'TRANSACTION_DATE',
 'CALC_BASE_WT_IN_GM',
 'DOSAGE_UNIT',
 'TRANSACTION_ID',
 'Product_Name',
 'Ingredient_Name',
 'Measure',
 'MME_Conversion_Factor',
 'Combined_Labeler_Name',
 'Revised_Company_Name',
 'Reporter_family',
 'dos_str']
def tsvtodfSPLIT(filename,hdrs,inserthdr=True):
    tlist=[]
    with open(filename) as tsvfile:
        #df=pd.DataFrame.from_csv(tsvfile)
        reader = csv.reader(tsvfile, delimiter='\t')
        for row in reader:
            tlist.append(row)

        dfp = pd.DataFrame(tlist)#
        #and make header names
        #hdrslist=dfp.T[0].tolist()
        if inserthdr:           
            dfp.columns=hdrs #make columns 
        dfp.dropna
        dfr = dfp.reset_index()
        return dfr

def ParseCSVFiles(Startfromcnt=-1):
    Pickupfromcount(Startfromcnt) #skip the first 285 records
    filelocs={} #dictionary of State:CSV filenames
    counter=0 #initialize counter
    Sourcedir="E:\\RiceProjectData\\DEAData" #directory containing the split files from the master DEA database
    Targetdir ="C:\\DEAData\\"                  #directory where to write data    

    for f in DEAFilelist:    

        basedir =Targetdir + f  
        print(basedir)
        counter+=1
        if not DEAFiledict[f]:
            df0 =tsvtodfSPLIT(basedir,Masterhdr,True) #reads in about 300,000 lines of master record  from a file that has been produced by a split

           #The reduced DataFrame set to produce from the master dataset
            dfRed = df0[[
             'REPORTER_NAME',   
             'BUYER_NAME',   
             'BUYER_ADDRESS1',
             'BUYER_STATE',
             'BUYER_COUNTY',   
             'BUYER_ZIP',
             'TRANSACTION_DATE',
             'CALC_BASE_WT_IN_GM',  
             'DRUG_NAME',
             'QUANTITY',
             'DOSAGE_UNIT',
             'dos_str'
             ]]
            df0.dropna
            #df0['QUANTITY'].astype(float)
            states=(dfRed.BUYER_STATE.unique())
            #outpath="E:\\RiceProjectData\\DEAData\\StateCSVfiles\\"
            outpath="C:\\DEAData\\StateCSVfiles\\"
            for s in states:
                dfs =dfRed.loc[dfRed['BUYER_STATE'] == s]
                if s in filelocs: #append data to existing CSV file i
                    curfile=filelocs[s] # Get the file name
                    #print(curfile + ' is being processd')
                    dfs.to_csv(curfile, mode='a',index=None,header=False) #append panda dataframe contents to CSV file
                    DEAFiledict.update({f:True}) #This file has been processed if this is set to True            
                else:  #Creates the filename for the first time and then copies the header 
                    tfile=outpath + s+'_DEA_data.csv'   #State     
                    filelocs.update( {s : tfile})
                    dfs.to_csv(tfile, index = None, header=True) #Don't forget to add '.csv' at the end of the path)
                    #print(tfile)
                #end if of filelocs (CSV files)
             #end For of States  
        #end if for DEADict
    #End For of DEAFileList            


 

### Process Data
Bring in as state and use Pivot Table to subcategorize

In [3]:
data = pd.read_csv("C:\\DEAData\\StateCSVfiles\\TX_DEA_data.csv" )
data.head()

,REPORTER_NAME,BUYER_NAME,BUYER_ADDRESS1,BUYER_STATE,BUYER_COUNTY,BUYER_ZIP,TRANSACTION_DATE,CALC_BASE_WT_IN_GM,DRUG_NAME,QUANTITY,DOSAGE_UNIT,dos_str
0,BELLCO DRUG CORP,LONGHORN DRUG CO,1711 S. HENDERSON BLVD.,TX,GREGG,75662,1112006,1.5135,HYDROCODONE,1.0,500.0,5.0
1,BELLCO DRUG CORP,MALLEY'S PHARMACY,MALLEY JOE DAVID,TX,JEFFERSON,77706,1042006,1.5135,HYDROCODONE,1.0,500.0,5.0
2,BELLCO DRUG CORP,MALLEY'S PHARMACY,MALLEY JOE DAVID,TX,JEFFERSON,77706,11022006,2.4216,HYDROCODONE,4.0,400.0,10.0
3,BELLCO DRUG CORP,MALLEY'S PHARMACY,MALLEY JOE DAVID,TX,JEFFERSON,77706,1042007,0.6054,HYDROCODONE,1.0,100.0,10.0
4,BELLCO DRUG CORP,MALLEY'S PHARMACY,MALLEY JOE DAVID,TX,JEFFERSON,77706,2142007,0.6054,HYDROCODONE,1.0,100.0,10.0
